In [1]:
!python -m pip install pulp

     ---------------------------------------- 17.7/17.7 MB 6.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pulp import *

In [3]:
# import of an instance
def import_instance(path):
  with open(path, 'r') as f:
    lines = f.readlines()

    m = int(lines[0].strip())
    n = int(lines[1].strip())
    l = [int(x) for x in lines[2].strip().split()]
    s = [int(x) for x in lines[3].strip().split()]

    dist_data = [line.strip().split() for line in lines][4:]
    D = [[int(x) for x in row] for row in dist_data]
  return m, n, l, s, D

In [5]:
#Define problem parameters
m, n, l, s, D = import_instance('inst09.dat')

#Create decision variables
choices = LpVariable.dicts("choices", (range(n+1), range(n+1),range(m)), cat=LpBinary,lowBound=0)
longest = LpVariable("longest", lowBound=0,cat=LpInteger)
c = LpVariable.dicts("c", (range(n+1),range(m)), lowBound=0,upBound=n,cat=LpInteger)
#Create problem instance
prob = LpProblem("MCP", LpMinimize)



#objective function
prob += longest

#ensure that longest is the max of all the distance
d = [lpSum([D[i][j] * choices[i][j][k] for i in range(n+1) for j in range(n+1) ]) for k in range(m)]

#ensure that longest is the maximum of the distance
for k in range(m):
  prob += longest >= d[k]

#ensure that there is only one edge starting from an item by limiting to 1 the number of 1 in a rows
for i in range(n):
  prob += lpSum([choices[i][j] for j in range(n+1) if i != j]) == 1

#ensure that there is only one edge going in an item by limiting to 1 the number of 1 in a columns
for i in range(n):
  prob += lpSum([choices[j][i] for j in range(n+1) if i != j]) == 1

#ensure that each courier start only one time from the origin
for k in range(m):
  prob += lpSum([choices[n][j][k] for j in range(n)]) == 1

#ensure that each courier ends only one time in the origin
for k in range(m):
  prob += lpSum([choices[j][n][k] for j in range(n)]) == 1

#no arc between the same item
for i in range(n+1):
    prob += lpSum(choices[i][i]) == 0


# Ensures that paths are connected
for k in range(m):
  for i in range(n+1):
    for j in range(n):
      if i != j:
        if i == n:
          prob += choices[i][j][k] <= lpSum([choices[j][h][k] for h in range(n+1) if h != j])
        else:
          prob += choices[i][j][k] <= lpSum([choices[j][h][k] for h in range(n+1) if h != j and h != i])


#ensure that the item are below the load limit
for k in range(m):
  prob += lpSum([s[i] * choices[i][j][k] for i in range(n) for j in range(n+1) ]) <= l[k]


#subtour elimination
for k in range(m):
  for i in range(n):
    for j in range(n):
      if i!=j:
        prob += c[i][k] + choices[i][j][k] <= c[j][k] + (n-1)*(1-choices[i][j][k])



time_limit_in_seconds = 60*5

prob.solve(PULP_CBC_CMD(msg=1, timeLimit=time_limit_in_seconds))


#Solve the problem

#Print the solution
print(LpStatus[prob.status])
print("Optimal solution:")
for k in range(m):
  for i in range(n+1):
    for j in range(n+1):
        if choices[i][j][k].varValue > 0:
            print(f"Courier {k+1} goes from {i+1} to {j+1}")
            #print(choices[i][j][k], "=", choices[i][j][k].varValue)
        else:
          #print(choices[i][j][k], "=", choices[i][j][k].varValue)
          pass
#print(prob)
print("Total distance:", value(prob.objective))

Optimal
Optimal solution:
Courier 1 goes from 3 to 5
Courier 1 goes from 5 to 7
Courier 1 goes from 7 to 14
Courier 1 goes from 14 to 3
Courier 2 goes from 6 to 14
Courier 2 goes from 14 to 6
Courier 3 goes from 12 to 14
Courier 3 goes from 14 to 12
Courier 4 goes from 8 to 14
Courier 4 goes from 14 to 8
Courier 5 goes from 9 to 14
Courier 5 goes from 14 to 9
Courier 6 goes from 1 to 14
Courier 6 goes from 14 to 1
Courier 7 goes from 2 to 14
Courier 7 goes from 14 to 2
Courier 8 goes from 10 to 14
Courier 8 goes from 14 to 10
Courier 9 goes from 4 to 13
Courier 9 goes from 13 to 14
Courier 9 goes from 14 to 4
Courier 10 goes from 11 to 14
Courier 10 goes from 14 to 11
Total distance: 436.0
